#**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-open-instruct](https://huggingface.co/TheBloke/open-llama-13b-open-instruct-GGML).

#**Step 1: Install All the Required Packages**

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 8.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 64.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-htalyfkr/overlay/local/bin
    changing mode of /tmp/pip-build-env-htalyfkr/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-htalyfkr/overlay/l

In [13]:
model_name_or_path = "TheBloke/open-llama-13b-open-instruct-GGML"
model_basename = "open-llama-13b-open-instruct.ggmlv3.q2_K.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama


#**Step 3: Download the Model**

In [14]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

(…)-llama-13b-open-instruct.ggmlv3.q2_K.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [15]:
# GPU
# lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [16]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Create a Prompt Template**

In [21]:
prompt = "print(x+y)"
prompt_template=f'''SYSTEM: Below is a line of python code that describes a task. Write one line of summary
that appropriately describes the task that the code is performing.
USER: {prompt}

ASSISTANT:
'''

#**Step 6: Generating the Response**

In [22]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [23]:
print(response)

{'id': 'cmpl-67fc35fa-3c39-4804-a8a6-7f025783c4ca', 'object': 'text_completion', 'created': 1706001735, 'model': '/root/.cache/huggingface/hub/models--TheBloke--open-llama-13b-open-instruct-GGML/snapshots/b896ba2e52979af6f83c3aecbfeca895e46ec8fb/open-llama-13b-open-instruct.ggmlv3.q2_K.bin', 'choices': [{'text': 'SYSTEM: Below is a line of python code that describes a task. Write one line of summary\nthat appropriately describes the task that the code is performing.\nUSER: print(x+y)\n\nASSISTANT:\nSummary: The code adds x and y to produce an output, which prints out the result of the addition operation', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 50, 'completion_tokens': 23, 'total_tokens': 73}}


In [24]:
print(response["choices"][0]["text"])

SYSTEM: Below is a line of python code that describes a task. Write one line of summary
that appropriately describes the task that the code is performing.
USER: print(x+y)

ASSISTANT:
Summary: The code adds x and y to produce an output, which prints out the result of the addition operation
